# <span style = 'color: red'>**Práctica uno.** Aprendizaje de máquina
# ***Regresión y clasificación***

- **Simón Cuartas Rendón**

En este *notebook* de Python se aborda la solución a los problemas de regresión y clasificación propuestos para la primera práctica del curso de **Aprendizaje de Máquina**.

A continuación, se abren los paquetes que se usarán tanto para los problemas de regresión como de clasificación.

**Importante.** Tanto para el problema de clasificación, como para el de regresión, se planteará un *pipeline* que permita automatizar el preprocesamiento de los datos, de tal forma que se puedan usar más adelante para cada uno de los modelos que se van a plantear.

In [1]:
#
# Apertura de paquetes genéricos
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Importación de matriz de confusión
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    cohen_kappa_score,
    classification_report,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)

In [2]:
#
# Configuración para no ver las salidas como HTML
pd.set_option('display.notebook_repr_html', False)

# <span style = 'color: green'>***1.***</span> Regresión

Para la práctica de regresión se empleará el conjunto de datos de `houseprices.csv`. Esta base contiene información del precio de venta de varias viviendas en el estado de California (Estados Unidos), e incluye un conjunto de covariables con las que se pretende poder predecir ese precio de la vivienda.

Así pues, se comenzará preprocesando la información y, a continuación, se pasará a ajustar varios modelos de regresión.

## <span style = 'color: cyan'>***1.1.***</span> Apertura del marco de datos y revisión inicial

In [3]:
#
# Apertura
direc_regr = '../Datos/houseprices.csv'
prices = pd.read_csv(direc_regr)

# Dimensión
print(f'El marco de datos tiene {prices.shape[0]} filas y {prices.shape[1]} columnas.')

El marco de datos tiene 1460 filas y 81 columnas.


In [4]:
#
# Vista de los primeros cinco registros
prices.head()

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [5]:
#
# Información de los tipos de datos
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

El listado anterior permite observar dos cosas:
- La primera es que los tipos de datos ya son los adecuados para cada variable de acuerdo con la documentación.
- Lo segundo es que existen variables que tienen poca información. Ejemplo de esto son variables como:
    - `PoolQC`, que se asocia con la calidad de la piscina, y sobre la cual solo existen siete registros.
    - `MasVnrType`, sobre el tipo de revestimiento de mampostería, que cuenta registros para una tercera parte aproximadamente de la cantidad de registros totales, en la medida que solo está disponible para 588 instancias.

En todo caso, la variable objetivo, `SalePrice`, tiene una nulidad del 0 \%.

Para continuar con el proceso de modelado, se van a conservar solo aquellas variables que tengan información disponible para al menos el 50 \% de los registros. A continuación se muestra la tasa de nulidad para cada característica:

In [6]:
#
# Tasa de nulidad por columna
tasa_nulidad = prices.isnull().sum() * 100 / len(prices)
tbl_tasa_nulidad = tasa_nulidad.apply(lambda x: f"{x:.1f}%")
print(tbl_tasa_nulidad.to_string())

Id                0.0%
MSSubClass        0.0%
MSZoning          0.0%
LotFrontage      17.7%
LotArea           0.0%
Street            0.0%
Alley            93.8%
LotShape          0.0%
LandContour       0.0%
Utilities         0.0%
LotConfig         0.0%
LandSlope         0.0%
Neighborhood      0.0%
Condition1        0.0%
Condition2        0.0%
BldgType          0.0%
HouseStyle        0.0%
OverallQual       0.0%
OverallCond       0.0%
YearBuilt         0.0%
YearRemodAdd      0.0%
RoofStyle         0.0%
RoofMatl          0.0%
Exterior1st       0.0%
Exterior2nd       0.0%
MasVnrType       59.7%
MasVnrArea        0.5%
ExterQual         0.0%
ExterCond         0.0%
Foundation        0.0%
BsmtQual          2.5%
BsmtCond          2.5%
BsmtExposure      2.6%
BsmtFinType1      2.5%
BsmtFinSF1        0.0%
BsmtFinType2      2.6%
BsmtFinSF2        0.0%
BsmtUnfSF         0.0%
TotalBsmtSF       0.0%
Heating           0.0%
HeatingQC         0.0%
CentralAir        0.0%
Electrical        0.1%
1stFlrSF   

In [7]:
#
# Alta nulidad
alta_nulidad = tasa_nulidad[tasa_nulidad > 50]
cols_alta_nulidad = alta_nulidad.index.tolist()
print(f'Columnas con nulidad superior al 50 %: {cols_alta_nulidad}')

Columnas con nulidad superior al 50 %: ['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature']


In [8]:
#
# Eliminación de columnas con alta nulidad
prices.drop(columns=cols_alta_nulidad, inplace=True)

print(f'El marco de datos conserva {prices.shape[0]} filas y {prices.shape[1]} columnas.')

El marco de datos conserva 1460 filas y 76 columnas.


Respecto a las filas, como se constató antes, la mayoría de columnas tienen tasas de nulidad del 0 \%, lo que hace que todas las observaciones tengan una cantidad de registros suficientes. En este orden de ideas, no se eliminarán filas.

In [9]:
#
# Resumen numérico
prices.describe()

                Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1201.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    70.049958   10516.828082     6.099315   
std     421.610009    42.300571    24.284752    9981.264932     1.382997   
min       1.000000    20.000000    21.000000    1300.000000     1.000000   
25%     365.750000    20.000000    59.000000    7553.500000     5.000000   
50%     730.500000    50.000000    69.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    80.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  ...  \
count  1460.000000  1460.000000   1460.000000  1452.000000  1460.000000  ...   
mean      5.575342  1971.267808   1984.865753   103.685262   443.639726  ...   
std       1.112799    30.202904     20.645407   181.066207   456.098091  ..

Se evidencia que existen variables continuas con rangos o soportes diferentes. Por ejemplo, el campo `GarageYrBlt` toma valores por el orden de los miles, toda vez que `EnclosedPorch` tomas valores que son de apenas unas cuantas decenas. En este orden de ideas, es necesario llevar a cabo un proceso de escalamiento que evite que el modelo se sesgue.

## <span style = 'color: cyan'>***1.2.***</span> Preprocesamiento de los datos

A continución, se definirá una tubería o *pipeline* con la cual se puede abordar el preprocesamiento de los datos. Para la conversión de los tipos de datos, se tomarán a las variables nominales como categóricas.

In [10]:
#
# Matriz de diseño y variable objetivo
X_regr = prices.drop(columns=['SalePrice'])
y_regr = prices['SalePrice']

In [11]:
#
# Selección de variables según su tipología
# IMPORTANTE: Solo incluir columnas que existen en X_regr
regr_num = X_regr.select_dtypes(include=['float64', 'int64']).columns.tolist()
regr_cat = X_regr.select_dtypes(include=['object', 'category']).columns.tolist()

print(
    f'Se agruparon un total de {len(regr_num)} variables numéricas y {len(regr_cat)} categóricas. \n',
    'Variables continuas:',
    regr_num,
    '\nVariables categóricas:',
    regr_cat,
    sep = '\n'
)

# Verificación adicional
print(f"\nVerificación:")
print(f"Total de columnas en X_regr: {X_regr.shape[1]}")
print(f"Variables numéricas + categóricas: {len(regr_num) + len(regr_cat)}")
print(f"¿Coinciden? {X_regr.shape[1] == len(regr_num) + len(regr_cat)}")


Se agruparon un total de 37 variables numéricas y 38 categóricas. 

Variables continuas:
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

Variables categóricas:
['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'K

In [12]:
#
# Transformaciones para las variables continuas
num_pipe_regr = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ])

# Transformaciones para las variables categóricas
cat_pipe_regr = Pipeline(steps=[
    ('onehot', OneHotEncoder(
        handle_unknown='ignore',
        drop='first')),
    ])

# Preprocesamiento
preprocesador_regr = ColumnTransformer(
    transformers=[
        ('num', num_pipe_regr, regr_num),
        ('cat', cat_pipe_regr, regr_cat),
    ],
    remainder='drop'
)
preprocesador_regr

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,feature_range,"(0, ...)"
,copy,True
,clip,False


In [13]:
# 
# Aplicación
X_regr_preproc = preprocesador_regr.fit_transform(X_regr)

# Convertir a matriz densa
import scipy.sparse
if scipy.sparse.issparse(X_regr_preproc):
    X_regr_preproc = X_regr_preproc.toarray()

# Conversión 
X_df_regr = pd.DataFrame(X_regr_preproc, columns=preprocesador_regr.get_feature_names_out())
X_df_regr.head()

    num__Id  num__MSSubClass  num__LotFrontage  num__LotArea  \
0  0.000000         0.235294          0.150685      0.033420   
1  0.000685         0.000000          0.202055      0.038795   
2  0.001371         0.235294          0.160959      0.046507   
3  0.002056         0.294118          0.133562      0.038561   
4  0.002742         0.235294          0.215753      0.060576   

   num__OverallQual  num__OverallCond  num__YearBuilt  num__YearRemodAdd  \
0          0.666667             0.500        0.949275           0.883333   
1          0.555556             0.875        0.753623           0.433333   
2          0.666667             0.500        0.934783           0.866667   
3          0.666667             0.500        0.311594           0.333333   
4          0.777778             0.500        0.927536           0.833333   

   num__MasVnrArea  num__BsmtFinSF1  ...  cat__SaleType_ConLI  \
0          0.12250         0.125089  ...                  0.0   
1          0.00000         0

Como se puede observar, 

# <span style = 'color: green'>***2.***</span> Clasificación

Se utilizará el marco de datos `HeartDisease`. El objetivo será entrenar modelos que permitan identificar si una persona está en riesgo o no de padecer una enfermedad cardíaca. Para ello, primero se va a evaluar el marco de datos y se realizará un preprocesamiento inicial de las variables regresoras.

## <span style = 'color: cyan'>***2.1.***</span> Apertura del marco de datos y revisión inicial

In [ ]:
#
# Apertura
direc_clasif = '../Datos/HeartDisease.csv'
hd = pd.read_csv(direc_clasif)

# Dimensión
print(f'El marco de datos tiene {hd.shape[0]} filas y {hd.shape[1]} columnas.')

In [ ]:
#
# Vista de los primeros cinco registros
hd.head()

In [ ]:
#
# Información de los tipos de datos
hd.info()

Nótese que todas las variables se han cargado como si fuerna cuantitativas; sin embargo, esto no es correcto en la medida que algunas de las variables son categóricas (por ejemplo: `sex` y `class`). En la etapa del preprocesamiento se corregirá esto para poder aplicar los tratamientos adecuados a cada una de las variables según su tipología.

In [ ]:
#
# Resumen numérico
hd.describe()

Se evidencia que existen variables continuas con rangos o soportes diferentes. Por ejemplo, el campo `resting bp s` toma valores por el orden de los cientos, mientras que `oldpeak` tomas valores que son de apenas unas cuantas centenas. En este orden de ideas, es necesario llevar a cabo un proceso de **escalamiento** que evite que el modelo se sesgue.

In [ ]:
#
# Verificación de valores nulos
hd.isna().any()

Para ninguna de las columnas se cuentan con instancias nulas, por lo que se van a usar todos los datos para entrenar el modelo. Ahora, se revisará si el marco de datos está desbalanceado.

In [ ]:
#
# Calcular frecuencia absoluta y relativa
conteo_obj = hd['target'].value_counts().sort_index()
relativo = hd['target'].value_counts(normalize=True).sort_index() * 100

# Crear figura y ejes
plt.figure(figsize=(6, 4))
ax = sns.barplot(x=conteo_obj.index, y=conteo_obj.values)

# Agregar etiquetas de porcentaje sobre las barras
for i, val in enumerate(relativo.values):
    ax.text(i, conteo_obj.values[i] + 5, f'{val:.1f}%', ha='center')

# Estética del gráfico
plt.title("Distribución de clases con porcentajes")
plt.xlabel("Clase")
plt.ylabel("Frecuencia")
plt.ylim(top=max(conteo_obj.values)*1.15)
plt.show()

Si bien etiqueta asociada a personas con mayor chance de desarrollar problemas cardíacos es más frecuente, ambas etiquetas difieren por menos de un 6 %, por lo que no se puede decir que este marco de datos esté desbalanceado. Así, se trabajará con estos mismos datos.

## <span style = 'color: cyan'>***2.2.***</span> Preprocesamiento de los datos

A continución, se definirá una tubería o *pipeline* con la cual se puede abordar el preprocesamiento de los datos. Para la conversión de los tipos de datos, se tomarán a las variables nominales como categóricas.

In [ ]:
#
# Matriz de diseño y variable objetivo
X_inic = hd.drop('target', axis=1)
y_inic = hd['target']

X_inic.head()

In [ ]:
#
# Cambio de tipología de datos para variables categóricas
cat_feats = ['sex', 'chest pain type', 'fasting blood sugar',
             'resting ecg', 'exercise angina', 'ST slope',
             'target']
X_inic[cat_feats] = X_inic[cat_feats].astype('category')

X_inic.info()

In [ ]:
#
# Selección de variables continuas y categóricas
cont_feats = X_inic.select_dtypes(include=[np.number]).columns.tolist()
cat_feats = X_inic.select_dtypes(include=['category']).columns.tolist()

print(
    'Variables continuas:',
    cont_feats,
    '\nVariables categóricas:',
    cat_feats,
    sep = '\n'
)

In [ ]:
#
# Transformaciones para las variables continuas
num_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ])

# Transformaciones para las variables categóricas
cat_pipe = Pipeline(steps=[
    ('onehot', OneHotEncoder(
        handle_unknown='ignore',
        drop='first')),
    ])

In [ ]:
#
# Preprocesamiento
preprocesador = ColumnTransformer(
    transformers=[
        ('num', num_pipe, cont_feats),
        ('cat', cat_pipe, cat_feats)
    ],
    remainder='drop',
)

preprocesador

In [ ]:
#
# Aplicación
X = preprocesador.fit_transform(X_inic)

# Conversión 
X_df = pd.DataFrame(X, columns=preprocesador.get_feature_names_out())
X_df.head()

Con esto listo, se puede proceder con el entrenamiento de los modelos.

In [ ]:
#
# Obtención de los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y_inic, test_size=0.25, random_state=42
)

## <span style = 'color: cyan'>***2.3.***</span> Clasificador de ***Bayes ingenuo***.

En este caso, se optimizará el parámetro de **suavizamiento**: `var_smoothing`. 

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
#
# Grilla de hiperparámetros
param_grid_nb = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

# Optimizador
grilla_nb = GridSearchCV(
    GaussianNB(),
    param_grid=param_grid_nb,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1
)

# Ajuste del modelo
grilla_nb.fit(X_train, y_train)

In [ ]:
#
# Extracción de los resultados de la optimización
result_nb = pd.DataFrame(grilla_nb.cv_results_)

# Extracción de parámetros y las exactitudes
var_smoothing_nb = result_nb['param_var_smoothing'].values
mean_test_score_nb = result_nb['mean_test_score'].values

In [ ]:
#
# Gráfico
plt.figure(figsize=(10, 6))
plt.plot(var_smoothing_nb, mean_test_score_nb, marker='o')
plt.xscale('log')
plt.xlabel('Factor de suavizado de varianza')
plt.ylabel('Exactitud media')
plt.title('Optimización del clasificador de Bayes ingenuo')

plt.scatter(
    var_smoothing_nb[np.argmax(mean_test_score_nb)],
    mean_test_score_nb.max(),
    color='red',
    s=120,              # tamaño más grande que los puntos normales
    edgecolors='red',
    linewidths=1.5,
    label=f'Max accuracy: {mean_test_score_nb.max():.2f}'
)

plt.legend()
plt.grid()
plt.show()

#
# Impresión de resultados
print(f"Mejor parámetro de suavizado de varianza: {grilla_nb.best_params_['var_smoothing']:.2f}")
print(f"Mejor exactitud: {grilla_nb.best_score_:.2f}")

In [ ]:
#
# Mejor modelo
mejor_nb = grilla_nb.best_estimator_

In [ ]:
def plot_confusion_and_roc(model, X_test, y_test, modelo=None, class_labels=None):

    # Predicciones
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # probabilidad clase positiva

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred, labels=class_labels)

    # Curva ROC 
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)

    # Figura con subplots 
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Matriz de confusión
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title('Matriz de confusión')
    axes[0].set_xlabel('Predicción')
    axes[0].set_ylabel('Real')

    # Curva ROC
    axes[1].plot(fpr, tpr, color='darkorange', label=f'AUC = {auc:.2f}')
    axes[1].plot([0, 1], [0, 1], color='navy', linestyle='--')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('Tasa de falsos positivos (FPR)')
    axes[1].set_ylabel('Tasa de verdaderos positivos (TPR)')
    axes[1].set_title('Curva ROC')
    axes[1].legend(loc='lower right')

    fig.suptitle(f"Desempeño del modelo: {modelo}", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

    # Métricas de evaluación"
    print("MÉTRICAS DE EVALUACIÓN ")
    print(f"Cohen's Kappa: {cohen_kappa_score(y_test, y_pred):.2f}")
    print(f"Exactitud: {accuracy_score(y_test, y_pred):.2f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.2f}")
    print(f"Precisión: {precision_score(y_test, y_pred):.2f}")
    print(f"Recall: {recall_score(y_test, y_pred):.2f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred):.2f}")
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred, target_names=class_labels))

In [ ]:
plot_confusion_and_roc(mejor_nb, X_test, y_test, modelo='Bayes ingenuo',)

En general, se puede ver que el modelo de Bayes ingenuo tiene un desempeño adecuado, en la medida que la mayoría de etiquetas para ambos casos están siendo correctamente clasificados. Lo que es posible anticipar de esta matriz de confusión es que hay un mayor error en la predicción cuando no hay riesgo de problemas cardíacos, en la medida que hay una mayor cantidad de equivocaciones a pesar de que esta etiqueta tiene ligeramente menos instancias. Esto se puede validar con el reporte de clasificación, donde se que para las distintas métricas, estas son peores para tal etiqueta.

## <span style = 'color: cyan'>***2.4.***</span> Clasificador de ***$k$ vecinos más cercanos***.

En este caso, se optimizarán los hiperparámetros de la cantidad de vecinos, los pesos, y el parámetro de Minkowski para tomar distancia Manhattan o euclidiana.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#
# Grilla de hiperparámetros
param_grid_knn = {
    'n_neighbors': np.arange(1, 40, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Optimizador
grilla_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid_knn,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1,
)

# Ajuste del modelo
grilla_knn.fit(X_train, y_train)

In [ ]:
#
# Extracción de los resultados de la optimización
result_knn = pd.DataFrame(grilla_knn.cv_results_)
results_knn = result_knn.sort_values(by='mean_test_score', ascending=False)

# Extracción de parámetros y las exactitudes
n_neighbors_knn = result_knn['param_n_neighbors'].astype(np.int64).values
mean_test_score_knn = result_knn['mean_test_score'].values  
metricas_knn = result_knn['param_metric'].unique()

In [ ]:
fig, axes = plt.subplots(1, len(metricas_knn), figsize=(12, 5), sharey=True)

for i, metrica in enumerate(metricas_knn):
    ax = axes[i]
    df_metrica = result_knn[result_knn['param_metric'] == metrica]

    # Inicializar listas para la búsqueda del mejor punto global en esta métrica
    mejores_k = []
    mejores_acc = []
    mejores_idx = []

    for peso in df_metrica['param_weights'].unique():
        df_sub = df_metrica[df_metrica['param_weights'] == peso]
        df_sub = df_sub.sort_values(by='param_n_neighbors')

        k_vals = df_sub['param_n_neighbors'].astype(int)
        acc_vals = df_sub['mean_test_score']

        ax.plot(k_vals, acc_vals, marker='o', label=f'weights={peso}')

        # Guardar los mejores candidatos
        idx_max = acc_vals.idxmax()
        mejores_k.append(k_vals.loc[idx_max])
        mejores_acc.append(acc_vals.loc[idx_max])
        mejores_idx.append(idx_max)

    # Elegir el mejor entre todos los pesos
    idx_best_in_metrica = np.argmax(mejores_acc)
    best_k = mejores_k[idx_best_in_metrica]
    best_acc = mejores_acc[idx_best_in_metrica]

    # Dibujar el círculo solo en el mejor global de esa métrica
    ax.scatter(best_k, best_acc, color='red', s=100, edgecolors='black', zorder=5)

    ax.set_title(f"Métrica: {metrica}")
    ax.set_xlabel("Número de vecinos (k)")
    if i == 0:
        ax.set_ylabel("Exactitud promedio (validación cruzada)")
    ax.grid(True)
    ax.legend()

    ax.set_xticks(sorted(df_metrica['param_n_neighbors'].unique()))

# Título general
fig.suptitle("Exactitud vs Número de Vecinos (KNN)\nComparación por métrica y tipo de weights", fontsize=14, y=1.08)
plt.tight_layout()
plt.show()

Como se puede observar, en el caso en el que se le otorga una mayor relevancia a las instancias *más cercanas* (y no de manera igualitaria), se alcanza una mejor exactitud. Además, cuando la métrica que se usa es la de Manhattan, se logra una mejor exactitud. En general, los mejor parámetros son:

In [ ]:
# Impresión de resultados
print(f"Mejor parámetro de suavizado de varianza: {grilla_knn.best_params_}")
print(f"Mejor exactitud: {grilla_knn.best_score_:.2f}")

In [ ]:
#
# Mejor modelo
mejor_knn = grilla_knn.best_estimator_

In [ ]:
plot_confusion_and_roc(mejor_knn, X_test, y_test, modelo='KNN',)

El modelo de *K vecinos más cercanos* tiene un desempeño superior al de K vecinos más cercanos, refléjandose en un aumento en todas las métricas y un mayor AUC. Además, si bien para los pacientes que en realidad no tienen riesgo de desarrollar enfermedades cardíacas aún se cumple que sus métricas son inferiores que para la otra clase, sí se evidencia un cierre en la brecha, pasadno por ejemplo de una diferencia de 7 % en el *recall*, a una brecha de solo 2 % en este caso.

## <span style = 'color: cyan'>***2.5.***</span> Clasificador de ***máquinas de soporte vectorial***.

Se van a definir varias grillas en función del tipo de kernel a proponer.

In [ ]:
from sklearn.svm import SVC

In [ ]:
#
# Grilla de hiperparámetros
param_grid_svc = [
    {
        'kernel': ['linear'],
        'C': [0.01, 0.1, 1, 10, 100]
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10, 100],
        'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto']
    },
    {
        'kernel': ['poly'],
        'C': [0.1, 1, 10],
        'degree': [2, 3, 4],
        'gamma': [0.01, 0.1, 'scale'],
        'coef0': [0, 0.5, 1]
    },
    {
        'kernel': ['sigmoid'],
        'C': [0.1, 1, 10],
        'gamma': [0.01, 0.1, 'scale'],
        'coef0': [0, 0.5, 1]
    }
]

# Optimizador
grilla_svc = GridSearchCV(
    SVC(probability=True, random_state=202506),
    param_grid=param_grid_svc,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1,
)

# Ajuste del modelo
grilla_svc.fit(X_train, y_train)

In [ ]:
#
# Mejor modelo
mejor_svc = grilla_svc.best_estimator_
mejor_svc_params = grilla_svc.best_params_

# Mejor modelo
mejor_svc_final = SVC(**mejor_svc_params, probability=True, random_state=202506)

# Reentrenar
mejor_svc_final.fit(X_train, y_train)

In [ ]:
plot_confusion_and_roc(mejor_svc_final, X_test, y_test, modelo='SVM',)

Nótese que el mejor kernel es el `rbf`. Este modelo presenta un mejor desempeño que el modelo de Bayes ingenuo; sin embargo, no logra superar al de $k$ vecinos más cercano. Además, amplía la brecha en el desempeño individual de las clases.

## <span style = 'color: cyan'>***2.6.***</span> Clasificador de ***redes neuronales***.

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
#
# Grilla de hiperparámetros
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 50, 25, 12)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'max_iter': [200, 300, 400],
    'random_state': [202506]
}

# Optimizador
grilla_mlp = GridSearchCV(
    MLPClassifier(),
    param_grid=param_grid_mlp,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1,
)

# Ajuste del modelo
grilla_mlp.fit(X_train, y_train)

In [ ]:
#
# Grilla de hiperparámetros
param_grid_knn = {
    'n_neighbors': np.arange(1, 40, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Optimizador
grilla_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid_knn,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1,
)

# Ajuste del modelo
grilla_knn.fit(X_train, y_train)

## <span style = 'color: cyan'>***2.7.***</span> Clasificador de ***ensamble:*** bosques aleatorios

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#
# Grilla de hiperparámetros
param_grid_rf = {
    'n_estimators': [100, 200, 300, ],
    'max_depth': [None, 10, 20, 30, ],
    'min_samples_split': [2, 5, 10, ],
    'min_samples_leaf': [1, 2, 4, ],
    'max_features': ['sqrt', 'log2', ],
    'random_state': [202506, ],
    'criterion': ['gini', 'entropy', ],
}

# Optimizador
grilla_rf = GridSearchCV(
    RandomForestClassifier(),
    param_grid=param_grid_rf,
    scoring='accuracy',
    cv=10,
    n_jobs=-1,
    verbose=1,
) 

# Ajuste del modelo
grilla_rf.fit(X_train, y_train)

In [ ]:
#
# Mejor modelo
mejor_rf = grilla_rf.best_estimator_
mejor_rf

In [ ]:
plot_confusion_and_roc(mejor_rf, X_test, y_test, modelo='SVM',)

En general, el modelo de bosques aleatorios tiene un desempeño muy bueno, similar al de los $k$ vecinos más cercanos; no obstante, al entrar al detalle de cada las dos clases por separado, se ve que la brecha es ligeralmente superior en este caso, pues si se toma el *recall* por ejemplo, pasa de un 2 % con KNN a un 3 % con bosques aleatorios.